In [3]:
import numpy as np
import torch
import modules.evaluate as evaluate

def continious_to_sparcity(my_array, top= 400):
    # Flatten the array to a 1D array
    flat_array = my_array.flatten()

    # Sort the flattened array in descending order
    sorted_indices = np.argsort(flat_array)[::-1]

    # Set the top 400 elements to 1 and the rest to 0
    flat_array[sorted_indices[:top]] = 1
    flat_array[sorted_indices[top:]] = 0

    # Reshape the modified 1D array back to the original shape
    result_array = flat_array.reshape(my_array.shape)
    return result_array

def continious_to_sparcity_cols(my_matrix, topRow= 8):

    result_matrix = np.zeros_like(my_matrix)
    indices = np.argsort(my_matrix, axis=1)[:, -topRow:]  # Get indices of top 8 elements in each row
    row_indices = np.arange(my_matrix.shape[0])[:, None]  # Row indices to broadcast
    result_matrix[row_indices, indices] = 1
    
    return result_matrix

modelLoads= ["AA", "AB", "BA", "BB", "infer2018"]
weightModels= ["degree", "gravity", "identical"]
randomGraphs= ["ER", "BA", "WS", "RGG"]
seeds= [10, 11, 12, 13, 14]
strains= [1, 2, 3, 4]
denses= [4, 6, 8]
nodes= [50, 100, 150]
intenses= [0, 1, 2]
identicalfs= [0.005, 0.01, 0.02]


commands1=set()
linuxThread= 3

In [10]:


#Performance on different dense
for randomGraph in randomGraphs:
    for seed in seeds:
        for dense in denses:
            command= f"python run.py --epoches {140000} --modelLoad AA --weightModel identical --randomGraph {randomGraph}\
              --seed {seed} --strains 4  --dense {dense}"
            commands1.add(command)
#Performance on different node number
for randomGraph in randomGraphs:
    for seed in seeds:
        for node in nodes:
            command= f"python run.py --epoches {140000} --modelLoad AA --weightModel identical --randomGraph {randomGraph}\
            --seed {seed} --strains 4  --n {node}"
            commands1.add(command)
#Performance on different intense
for randomGraph in randomGraphs:
    for seed in seeds:
        for intsense in intenses:
            command= f"python run.py --epoches {140000} --modelLoad AA --weightModel identical --randomGraph {randomGraph}\
              --seed {seed} --strains 4  --intense {intsense}"
            commands1.add(command)
#Performance on different identical float
for randomGraph in randomGraphs:
    for seed in seeds:
        for identicalf in identicalfs:
            command= f"python run.py --epoches {140000} --modelLoad AA --weightModel identical --randomGraph {randomGraph}\
              --seed {seed} --strains 4  --identicalf {identicalf}"
            commands1.add(command)
count=0
with open("myCommands.bat", 'w') as file:
    # Write each element on a new line
    for i, string in enumerate(commands1):
        file.write(f"echo \"{string} --CMDprogress_{i}_{len(commands1)-1}\"\n")
        file.write(f"{string} --CMDprogress_{i}_{len(commands1)-1}\n")
        
count=0
with open("myCommands.sh", 'w') as file:
    # Write each element on a new line
    for i, string in enumerate(commands1):
        file.write(f"echo \"{string} --CMDprogress_{i}_{len(commands1)-1}\"\n")
        file.write(f"{string} --CMDprogress_{i}_{len(commands1)-1} &\n")
        count+=1
        if count%linuxThread==0:
            file.write("wait\n")
    if count%linuxThread !=0:
        file.write("wait\n")
# echo "python run.py --epoches 140000 --modelLoad AA --weightModel identical --randomGraph BA --seed 14 --strains 2 --CMDprogress_78_79"
# python run.py --epoches 140000 --modelLoad AA --weightModel identical --randomGraph BA --seed 14 --strains 2 --CMDprogress_78_79 &
# echo "python run.py --epoches 140000 --modelLoad AA --weightModel identical --randomGraph BA --seed 14 --strains 3 --CMDprogress_79_79"
# python run.py --epoches 140000 --modelLoad AA --weightModel identical --randomGraph BA --seed 14 --strains 3 --CMDprogress_79_79 &
        

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import modules.plotGraph as plotGraph
import networkx as nx
from matplotlib.colors import Normalize

IMatrixNp= np.eye(50)

maxValue= 0
def load_npz_matrix(fileName, maxValue):
    npzfile = np.load(fileName)
    # Access the arrays in the NPZ file
    Aw = npzfile['A']
    PreA = npzfile['Apre']
    tempMax= np.max(PreA)
    if maxValue<tempMax:
        maxValue=tempMax
    return Aw, PreA, maxValue

PreAs=[]
Aw= None
randomGraph= "RGG"
seed= 14
for i in range(4):
    fileName= f"results/AA/AA_{randomGraph}_identical_{seed}_{i+1}_0_8_50_{0.01*1000}_140000.npz"
    Aw, PreA, maxValue= load_npz_matrix(fileName, maxValue)
    links= int((np.sum(Aw))/0.01)
    Awsp= continious_to_sparcity(Aw, links)


    PreAs.append(PreA)

PreAsp= [continious_to_sparcity(i, links) for i in PreAs]
campThis= "viridis"

plotGraph.plot_adjacenty(Aw, PreAs)
plotGraph.plot_adjacenty(Awsp, PreAsp)
print(maxValue)

0.025998432


In [15]:
links= int((np.sum(Aw))/0.01)
print(links)



As= continious_to_sparcity(Aw, links)
graphA = nx.from_numpy_matrix(As)
plotGraph.plot_spring_layout(graphA)

PreAsp= continious_to_sparcity(PreAs[0], links)
graphPreA = nx.from_numpy_matrix(PreAsp)
plotGraph.plot_spring_layout(graphPreA)

PreAsp= continious_to_sparcity(PreAs[1], links)
graphPreA = nx.from_numpy_matrix(PreAsp)
plotGraph.plot_spring_layout(graphPreA)

PreAsp= continious_to_sparcity(PreAs[2], links)
graphPreA = nx.from_numpy_matrix(PreAsp)
plotGraph.plot_spring_layout(graphPreA)

PreAsp= continious_to_sparcity(PreAs[3], links)
graphPreA = nx.from_numpy_matrix(PreAsp)
plotGraph.plot_spring_layout(graphPreA)

799
